In [24]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.utils import resample
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.metrics import classification_report
%matplotlib inline

In [25]:
wisc= pd.read_csv('C:/Python/hw2/wisc_bc_data.csv' , encoding='utf-8')
del wisc['id']

In [26]:
wisc

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,B,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,0.03700,0.1959,...,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
1,B,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,0.02642,0.1922,...,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
2,B,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,0.02480,0.1714,...,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
3,B,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,0.04796,0.1771,...,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
4,B,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,0.02657,0.1721,...,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,B,13.17,18.22,84.28,537.3,0.07466,0.05994,0.04859,0.02870,0.1454,...,14.90,23.89,95.10,687.6,0.1282,0.1965,0.18760,0.10450,0.2235,0.06925
565,B,10.26,14.71,66.20,321.6,0.09882,0.09159,0.03581,0.02037,0.1633,...,10.88,19.48,70.89,357.1,0.1360,0.1636,0.07162,0.04074,0.2434,0.08488
566,M,15.28,22.41,98.92,710.6,0.09057,0.10520,0.05375,0.03263,0.1727,...,17.80,28.03,113.80,973.1,0.1301,0.3299,0.36300,0.12260,0.3175,0.09772
567,B,14.53,13.98,93.86,644.2,0.10990,0.09242,0.06895,0.06495,0.1650,...,15.80,16.93,103.10,749.9,0.1347,0.1478,0.13730,0.10690,0.2606,0.07810


In [27]:
X = np.array(wisc.iloc[:,1:])
y = np.array(wisc.diagnosis)
print('클래스 레이블:',np.unique(y))

클래스 레이블: ['B' 'M']


In [28]:
le = LabelEncoder()
y = le.fit_transform(y)

In [29]:
X_train, X_test, y_train, y_test =train_test_split(
    X,y,random_state=1,test_size=0.3, stratify=y)

In [30]:
sc=StandardScaler()
sc.fit(X_train)
X_train_std=sc.transform(X_train)
X_test_std=sc.transform(X_test)

1. 다음 방법들을 사용하여 분류 문제를 풀어보고 서로 비교하시오.
 a. 로지스틱 회귀방법
 b. SVM
 c. 랜덤 포레스트
 d. 배깅과 에이다부스트
 e. k-NN
 
 * 그리드 서치를 활용한 초매개변수 튜닝을 실시하시오.
 * 교차타당성 (중첩 교차 검증)을 실시하시오
 * 비교시 분류기를 평가하는 각종 지표 (혼돈행렬, 오분류율, 정확도, 민감도, 특이도, ROC 곡선, AUC, F1 score,     recall, 정밀도 등)을 사용하시오. 

# LogisticRegression 

In [31]:
# 그리드서치
LR= LogisticRegression(random_state=1)
param_grid = {
    'C':[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}

lr = GridSearchCV(LR,                        # 모델 객체
                  param_grid=param_grid,     # 하이퍼파라미터 목록
                  scoring='accuracy',
                  cv = 2,
                  n_jobs=-1)
lr.fit(X_train_std, y_train)
print(lr.best_params_)
print(lr.best_estimator_)
print(lr.best_score_)

{'C': 1.0}
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
0.9723618090452262


In [32]:
LR.fit(X_train_std,y_train)
LR.score(X_test_std,y_test)

0.9824561403508771

In [33]:
# 중첩교차검증
scores = cross_val_score(lr, X_train_std, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

CV accuracy: 0.978 +/- 0.020


In [34]:
y_pred_lr = lr.predict(X_test_std)
print('logistic regression Accuracy: %.3f' % accuracy_score(y_test, y_pred_lr))

logistic regression Accuracy: 0.982


In [35]:
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       107
           1       0.98      0.97      0.98        64

    accuracy                           0.98       171
   macro avg       0.98      0.98      0.98       171
weighted avg       0.98      0.98      0.98       171



In [36]:
print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred_lr))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred_lr))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred_lr))

Precision: 0.984
Recall: 0.969
F1: 0.976


# svm

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

pipe_svc = make_pipeline(StandardScaler(),
                         SVC(random_state=1))

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]

svm = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=2,
                  n_jobs=-1)
svm = svm.fit(X_train, y_train)
print(svm.best_params_)
print(svm.best_estimator_)
print(svm.best_score_)

{'svc__C': 0.1, 'svc__kernel': 'linear'}
Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='linear', max_iter=-1,
                     probability=False, random_state=1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)
0.9723618090452262


In [38]:
pipe_svc.fit(X_train, y_train)
pipe_svc.score(X_test,y_test)

0.9707602339181286

In [39]:
# 중첩교차검증
scores = cross_val_score(svm, X_train, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

CV accuracy: 0.970 +/- 0.017


In [40]:
y_pred_svm = svm.predict(X_test)
print('svm Accuracy: %.3f' % accuracy_score(y_test, y_pred_svm))

svm Accuracy: 0.988


In [41]:
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       107
           1       0.98      0.98      0.98        64

    accuracy                           0.99       171
   macro avg       0.99      0.99      0.99       171
weighted avg       0.99      0.99      0.99       171



In [42]:
print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred_svm))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred_svm))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred_svm))

Precision: 0.984
Recall: 0.984
F1: 0.984


# randomforest

In [43]:
RF =  RandomForestClassifier(random_state=1)
param_grid = {
    'n_estimators':[100,200,300,400,500],
    'max_depth':[3,4,5,6,7],
    'max_leaf_nodes':[3,5,7,9,10],
    'criterion':['entropy', 'jini']
}

rf = GridSearchCV(RF,                        # 모델 객체
                  param_grid=param_grid,     # 하이퍼파라미터 목록
                  scoring='accuracy',
                  cv = 2,
                  n_jobs=-1)
rf.fit(X_train, y_train)
print(rf.best_params_)
print(rf.best_estimator_)
print(rf.best_score_)

KeyboardInterrupt: 

In [ ]:
# 중첩교차검증
scores = cross_val_score(rf, X_train, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

In [ ]:
y_pred_rf = rf.predict(X_test)
print('random forest Accuracy: %.3f' % accuracy_score(y_test, y_pred_rf))

In [ ]:
print(classification_report(y_test, y_pred_rf))

In [ ]:
print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred_rf))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred_rf))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred_rf))

# bagging

In [ ]:
BA =  BaggingClassifier(base_estimator=RF,random_state=1)
param_grid = {
    'max_samples':[7,9,11],
    'max_features':[7,9,11]
}

ba = GridSearchCV(BA,                        # 모델 객체
                  param_grid=param_grid,     # 하이퍼파라미터 목록
                  scoring='accuracy',
                  cv = 3,
                  n_jobs=-1)
ba.fit(X_train, y_train)
print(ba.best_params_)
print(ba.best_estimator_)
print(ba.best_score_)

In [ ]:
# 중첩교차검증
scores = cross_val_score(ba ,X_train, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

In [ ]:
y_pred_ba = ba.predict(X_test)
print('bagging random forest Accuracy: %.3f' % accuracy_score(y_test, y_pred_ba))

In [ ]:
print(classification_report(y_test, y_pred_ba))

In [ ]:
print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred_ba))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred_ba))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred_ba))

# ada

In [ ]:
ADA = AdaBoostClassifier(base_estimator=RF, random_state=1 )
param_grid = {
     'n_estimators': [200,300,400],
     'learning_rate': [ 0.05, 0.1, 1],
 }

ada = GridSearchCV(ADA,                        # 모델 객체
                  param_grid=param_grid,     # 하이퍼파라미터 목록
                  scoring='accuracy',
                  cv = 3,
                  n_jobs=-1)
ada.fit(X_train, y_train)
print(ada.best_params_)
print(ada.best_estimator_)
print(ada.best_score_)

In [ ]:
# 중첩교차검증
scores = cross_val_score(ada ,X_train, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

In [ ]:
y_pred_ada = ada.predict(X_test)
print('bagging random forest Accuracy: %.3f' % accuracy_score(y_test, y_pred_ada))

In [ ]:
print(classification_report(y_test, y_pred_ada))

In [ ]:
print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred_ada))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred_ada))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred_ada))

# knn

In [ ]:
#KNN =  KNeighborsClassifier(p=2, metric='minkowski')
KNN =  KNeighborsClassifier()
param_grid = {
    'n_neighbors':[1,2,3,4,5,6,7,8,9,10]
}

knn = GridSearchCV(KNN,                        # 모델 객체
                  param_grid=param_grid,     # 하이퍼파라미터 목록
                  scoring='accuracy',
                  cv = 3,
                  n_jobs=-1)
knn.fit(X_train_std, y_train)
print(knn.best_params_)
print(knn.best_estimator_)
print(knn.best_score_)

In [ ]:
# 중첩교차검증
scores = cross_val_score(knn ,X_train, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

In [ ]:
y_pred_knn = knn.predict(X_test_std)
print('bagging random forest Accuracy: %.3f' % accuracy_score(y_test, y_pred_knn))

In [ ]:
print(classification_report(y_test, y_pred_knn))

In [ ]:
print('Precision: %.3f' % precision_score(y_true=y_test, y_pred=y_pred_knn))
print('Recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred_knn))
print('F1: %.3f' % f1_score(y_true=y_test, y_pred=y_pred_knn))

In [ ]:
print('lr Misclassified examples: %d' % (y_test != y_pred_lr).sum())
print('svm Misclassified examples: %d' % (y_test != y_pred_svm).sum())
print('rf Misclassified examples: %d' % (y_test != y_pred_rf).sum())
print('ba Misclassified examples: %d' % (y_test != y_pred_ba).sum())
#print('Misclassified examples: %d' % (y_test != y_pred_ada).sum())
print('knn Misclassified examples: %d' % (y_test != y_pred_knn).sum())

In [ ]:
confmat1 = confusion_matrix(y_true=y_test, y_pred=y_pred_lr)
confmat2 = confusion_matrix(y_true=y_test, y_pred=y_pred_svm)
confmat3 = confusion_matrix(y_true=y_test, y_pred=y_pred_rf)
confmat4 = confusion_matrix(y_true=y_test, y_pred=y_pred_ba)
confmat5 = confusion_matrix(y_true=y_test, y_pred=y_pred_ada)
confmat6 = confusion_matrix(y_true=y_test, y_pred=y_pred_knn)
print(confmat1)
print(confmat2)
print(confmat3)
print(confmat4)
print(confmat5)
print(confmat6)

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat1, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat1.shape[0]):
    for j in range(confmat1.shape[1]):
        ax.text(x=j, y=i, s=confmat1[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat2, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat2.shape[0]):
    for j in range(confmat2.shape[1]):
        ax.text(x=j, y=i, s=confmat2[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat3, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat3.shape[0]):
    for j in range(confmat3.shape[1]):
        ax.text(x=j, y=i, s=confmat3[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat4, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat4.shape[0]):
    for j in range(confmat4.shape[1]):
        ax.text(x=j, y=i, s=confmat4[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat5, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat5.shape[0]):
    for j in range(confmat5.shape[1]):
        ax.text(x=j, y=i, s=confmat5[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat6, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat6.shape[0]):
    for j in range(confmat6.shape[1]):
        ax.text(x=j, y=i, s=confmat6[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

In [ ]:
probas1_ = lr.fit(X_train_std, y_train).predict_proba(X_test_std)
#probas2_ = svm.fit(X_train_std, y_train).predict_proba(X_test_std)
probas3_ = rf.fit(X_train, y_train).predict_proba(X_test)
probas4_ = ba.fit(X_train, y_train).predict_proba(X_test)
#probas5_ = ada.fit(X_train, y_train).predict_proba(X_test)
probas6_ = knn.fit(X_train_std, y_train).predict_proba(X_test_std)

In [ ]:
fpr1, tpr1, thresholds1 = roc_curve(y_test, probas1_[:, 1])
roc_auc_model1 = auc(fpr1, tpr1)
#fpr2, tpr2, thresholds2 = roc_curve(y_test, probas2_[:, 1])
#roc_auc_model2 = auc(fpr2, tpr2)
fpr3, tpr3, thresholds3 = roc_curve(y_test, probas3_[:, 1])
roc_auc_model3 = auc(fpr3, tpr3)
fpr4, tpr4, thresholds4 = roc_curve(y_test, probas4_[:, 1])
roc_auc_model4 = auc(fpr4, tpr4)
#fpr5, tpr5, tresholds5 = roc_curve(y_test, probas5_[:, 1])
#roc_auc_model5 = auc(fpr5, tpr5)
fpr6, tpr6, thresholds6 = roc_curve(y_test, probas6_[:, 1])
roc_auc_model6 = auc(fpr6, tpr6)
print("AUC for Logistic Regression Model : ",roc_auc_model1)
#print("AUC for SVM Model:", roc_auc_model2)
print("AUC for Random Forest Model :" ,roc_auc_model3)
print("AUC for bagging model :", roc_auc_model4)
#print("AUC for ada model :", roc_auc_model5)
print("AUC for knn model :", roc_auc_model6)

In [ ]:
plt.clf()
plt.plot(fpr1, tpr1, label='Logistic Model (area = %0.2f)' % roc_auc_model1)
#plt.plot(fpr2, tpr2, label='SVC Model (area = %0.2f)' % roc_auc_model2)
plt.plot(fpr3, tpr3, label='Random Forest Model (area = %0.2f)' % roc_auc_model3)
plt.plot(fpr4, tpr4, label='bagging Model (area = %0.2f)' % roc_auc_model4)
#plt.plot(fpr5, tpr5, label='ada Model (area = %0.2f)' % roc_auc_model5)
plt.plot(fpr6, tpr6, label='knn Model (area = %0.2f)' % roc_auc_model6)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiverrating characteristic example')
plt.legend(loc="lower right")
plt.show()

# 2. 로지스틱 회귀방법에서 L1 규제를 통한 특성 선택을 시도하고, 랜덤 포레스트를 사용하여 특성 중요도를 계산하고 서로 비교하여라.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.subplot(111)
    
colors = ['blue', 'green', 'red', 'cyan', 
          'magenta', 'yellow', 'black', 
          'pink', 'lightgreen', 'lightblue', 
          'gray', 'indigo', 'orange','coral','gold','plum',
          'silver', 'rosybrown', 'darksalmon', 'sienna', 
          'bisque','tan', 'moccasin', 
          'navy', 'm', 'palevioletred', 
          'deepskyblue', 'chartreuse', 'c','lavender','peru']

weights, params = [], []
for c in np.arange(-4., 6.):
    lr2 = LogisticRegression(penalty='l1', C=10.**c, solver='liblinear', 
                            multi_class='ovr', random_state=0)
    lr2.fit(X_train_std, y_train)
    weights.append(lr2.coef_[0])
    params.append(10**c)

weights = np.array(weights)

for column, color in zip(range(weights.shape[1]), colors):
    plt.plot(params, weights[:, column],
             label=wisc.columns[column + 1],
             color=color)
plt.axhline(0, color='black', linestyle='--', linewidth=3)
plt.xlim([10**(-5), 10**5])
plt.ylabel('weight coefficient')
plt.xlabel('C')
plt.xscale('log')
plt.legend(loc='upper left')
ax.legend(loc='upper center', 
          bbox_to_anchor=(1.38, 1.03),
          ncol=1, fancybox=True)
#plt.savefig('images/04_07.png', dpi=300, 
#            bbox_inches='tight', pad_inches=0.2)
plt.show()

# 랜덤포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

feat_labels = wisc.columns[1:]

forest = RandomForestClassifier(n_estimators=500,
                                random_state=1)

forest.fit(X_train, y_train)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

plt.title('Feature Importance')
plt.bar(range(X_train.shape[1]), 
        importances[indices],
        align='center')

plt.xticks(range(X_train.shape[1]), 
           feat_labels[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
#plt.savefig('images/04_09.png', dpi=300)
plt.show()

In [ ]:
from sklearn.feature_selection import SelectFromModel

sfm = SelectFromModel(forest, threshold=0.08, prefit=True)
X_selected = sfm.transform(X_train)
print('Number of features that meet this threshold criterion:', 
      X_selected.shape[1])

In [ ]:
for f in range(X_selected.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

# winedata

In [44]:
wine= pd.read_csv('C:/Python/hw2/wine.csv' , encoding='utf-8')

In [45]:
wine

,Alcohol,Malic.Acid,Ash,Alcalinity.of.Ash,Magnesium,Total.Phenols,Flavanoids,Nonflavanoid.Phenols,Proanthocyanins,Color.Intensity,Hue,OD280.OD315.of.Diluted.Wines,Proline,Cultivar
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,1
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,1
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,1
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,1
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,3
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,3
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,3
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,3


In [46]:
X = np.array(wine.iloc[:,:13])
y = np.array(wine.Cultivar)
print('클래스 레이블:',np.unique(y))

클래스 레이블: [1 2 3]


In [47]:
X_train, X_test, y_train, y_test =train_test_split(
    X,y,random_state=1,test_size=0.3, stratify=y)

In [48]:
sc=StandardScaler()
sc.fit(X_train)
X_train_std=sc.transform(X_train)
X_test_std=sc.transform(X_test)

# logistic regression

In [49]:
# 그리드서치
LR= LogisticRegression(random_state=1)
param_grid = {
    'C':[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
}

lr = GridSearchCV(LR,                        # 모델 객체
                  param_grid=param_grid,     # 하이퍼파라미터 목록
                  scoring='accuracy',
                  cv = 3,
                  n_jobs=-1)
lr.fit(X_train_std, y_train)
print(lr.best_params_)
print(lr.best_estimator_)
print(lr.best_score_)

{'C': 0.01}
LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
0.983933410762679


In [50]:
# 중첩교차검증
scores = cross_val_score(lr, X_train_std, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

CV accuracy: 0.984 +/- 0.020


In [51]:
y_pred_lr = lr.predict(X_test_std)
print('logistic regression Accuracy: %.3f' % accuracy_score(y_test, y_pred_lr))

logistic regression Accuracy: 0.981


In [52]:
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           1       0.95      1.00      0.97        18
           2       1.00      0.95      0.98        21
           3       1.00      1.00      1.00        15

    accuracy                           0.98        54
   macro avg       0.98      0.98      0.98        54
weighted avg       0.98      0.98      0.98        54



# SVM

In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

pipe_svc = make_pipeline(StandardScaler(),
                         SVC(random_state=1))

param_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]

param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]

svm = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=10,
                  n_jobs=-1)
svm = svm.fit(X_train, y_train)
print(svm.best_params_)
print(svm.best_estimator_)
print(svm.best_score_)

{'svc__C': 0.1, 'svc__kernel': 'linear'}
Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='linear', max_iter=-1,
                     probability=False, random_state=1, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)
0.9923076923076923


In [54]:
# 중첩교차검증
scores = cross_val_score(svm, X_train, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

CV accuracy: 0.968 +/- 0.030


In [55]:
y_pred_svm = svm.predict(X_test_std)
print('svm Accuracy: %.3f' % accuracy_score(y_test, y_pred_svm))

svm Accuracy: 0.389


In [56]:
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        18
           2       0.39      1.00      0.56        21
           3       0.00      0.00      0.00        15

    accuracy                           0.39        54
   macro avg       0.13      0.33      0.19        54
weighted avg       0.15      0.39      0.22        54



C:\Users\ddhhd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# random forest

In [57]:
RF =  RandomForestClassifier(random_state=1)
param_grid = {
    'max_depth':[3,4,5,6,7],
    'max_leaf_nodes':[3,5,7,9,10],
    'criterion':['entropy', 'jini']
}

rf = GridSearchCV(RF,                        # 모델 객체
                  param_grid=param_grid,     # 하이퍼파라미터 목록
                  scoring='accuracy',
                  cv = 3,
                  n_jobs=-1)
rf.fit(X_train, y_train)
print(rf.best_params_)
print(rf.best_estimator_)
print(rf.best_score_)

{'criterion': 'entropy', 'max_depth': 3, 'max_leaf_nodes': 7}
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=3, max_features='auto',
                       max_leaf_nodes=7, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)
0.951413085559427


In [58]:
# 중첩교차검증
scores = cross_val_score(rf, X_train, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

CV accuracy: 0.976 +/- 0.020


In [59]:
y_pred_rf = rf.predict(X_test)
print('random forest Accuracy: %.3f' % accuracy_score(y_test, y_pred_rf))

random forest Accuracy: 1.000


In [60]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        18
           2       1.00      1.00      1.00        21
           3       1.00      1.00      1.00        15

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54



# bagging

In [61]:
BA =  BaggingClassifier(base_estimator=RF,random_state=1)
param_grid = {
    'max_samples':[5,7,9],
    'max_features':[5,7,9]
}

ba = GridSearchCV(BA,                        # 모델 객체
                  param_grid=param_grid,     # 하이퍼파라미터 목록
                  scoring='accuracy',
                  cv = 3,
                  n_jobs=-1)
ba.fit(X_train, y_train)
print(ba.best_params_)
print(ba.best_estimator_)
print(ba.best_score_)

{'max_features': 9, 'max_samples': 9}
BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        max_samples=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                     

In [62]:
# 중첩교차검증
scores = cross_val_score(ba ,X_train, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

CV accuracy: 0.903 +/- 0.032


In [63]:
y_pred_ba = ba.predict(X_test)
print(classification_report(y_test, y_pred_ba))

              precision    recall  f1-score   support

           1       0.95      1.00      0.97        18
           2       1.00      0.95      0.98        21
           3       1.00      1.00      1.00        15

    accuracy                           0.98        54
   macro avg       0.98      0.98      0.98        54
weighted avg       0.98      0.98      0.98        54



# ada

In [ ]:
ada = AdaBoostClassifier(base_estimator=RF, random_state=1 )
param_grid = {
     'n_estimators': [200,300,400],
     'learning_rate': [0.01, 0.05, 0.1, 1],
 }

ada = GridSearchCV(ADA,                        # 모델 객체
                  param_grid=param_grid,     # 하이퍼파라미터 목록
                  scoring='accuracy',
                  cv = 3,
                  n_jobs=-1)
ada.fit(X_train, y_train)
print(ada.best_params_)
print(ada.best_estimator_)
print(ada.best_score_)

In [ ]:
# 중첩교차검증
scores = cross_val_score(ada ,X_train, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

In [ ]:
y_pred_ada = ada.predict(X_test)
print('ADA  Accuracy: %.3f' % accuracy_score(y_test, y_pred_ada))

In [ ]:
print(classification_report(y_test, y_pred_ada))

# knn

In [ ]:
#KNN =  KNeighborsClassifier(p=2, metric='minkowski')
KNN =  KNeighborsClassifier()
param_grid = {
    'n_neighbors':[1,2,3,4,5,6,7,8,9,10]
}

knn = GridSearchCV(KNN,                        # 모델 객체
                  param_grid=param_grid,     # 하이퍼파라미터 목록
                  scoring='accuracy',
                  cv = 3,
                  n_jobs=-1)
knn.fit(X_train_std, y_train)
print(knn.best_params_)
print(knn.best_estimator_)
print(knn.best_score_)

In [ ]:
# 중첩교차검증
scores = cross_val_score(knn ,X_train, y_train, 
                         scoring='accuracy', cv=5)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

In [ ]:
y_pred_knn = knn.predict(X_test_std)
print('knn Accuracy: %.3f' % accuracy_score(y_test, y_pred_knn))

In [ ]:
print(classification_report(y_test, y_pred_knn))

### misclassified examples, accuracy

In [ ]:
y_pred1 = lr.predict(X_test_std)
y_pred2 = svm.predict(X_test)
y_pred3 = rf.predict(X_test)
y_pred4 = ba.predict(X_test)
y_pred5 = ada.predict(X_test)
y_pred6 = knn.predict(X_test_std)

print('Misclassified examples: %d' % (y_test != y_pred1).sum())
print('Misclassified examples: %d' % (y_test != y_pred2).sum())
print('Misclassified examples: %d' % (y_test != y_pred3).sum())
print('Misclassified examples: %d' % (y_test != y_pred4).sum())
print('Misclassified examples: %d' % (y_test != y_pred5).sum())
print('Misclassified examples: %d' % (y_test != y_pred6).sum())

In [ ]:
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred1))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred2))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred3))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred4))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred5))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred6))

# 혼돈행렬

In [ ]:
confmat1 = confusion_matrix(y_true=y_test, y_pred=y_pred1)
confmat2 = confusion_matrix(y_true=y_test, y_pred=y_pred2)
confmat3 = confusion_matrix(y_true=y_test, y_pred=y_pred3)
confmat4 = confusion_matrix(y_true=y_test, y_pred=y_pred4)
confmat5 = confusion_matrix(y_true=y_test, y_pred=y_pred5)
confmat6 = confusion_matrix(y_true=y_test, y_pred=y_pred6)
print(confmat1)
print(confmat2)
print(confmat3)
print(confmat4)
print(confmat5)
print(confmat6)

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat1, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat1.shape[0]):
    for j in range(confmat1.shape[1]):
        ax.text(x=j, y=i, s=confmat2[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat2, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat2.shape[0]):
    for j in range(confmat2.shape[1]):
        ax.text(x=j, y=i, s=confmat2[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat3, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat3.shape[0]):
    for j in range(confmat3.shape[1]):
        ax.text(x=j, y=i, s=confmat2[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat4, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat4.shape[0]):
    for j in range(confmat4.shape[1]):
        ax.text(x=j, y=i, s=confmat2[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat5, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat5.shape[0]):
    for j in range(confmat5.shape[1]):
        ax.text(x=j, y=i, s=confmat2[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.matshow(confmat6, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat6.shape[0]):
    for j in range(confmat6.shape[1]):
        ax.text(x=j, y=i, s=confmat2[i, j], va='center', ha='center')

plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.tight_layout()
#plt.savefig('images/06_09.png', dpi=300)
plt.show()

# auc roc

In [ ]:
probas1_ = lr.fit(X_train_std, y_train).predict_proba(X_test_std)
#robas2_ = svm.fit(X_train_std, y_train).predict_proba(X_test_std)
probas3_ = rf.fit(X_train, y_train).predict_proba(X_test)
probas4_ = ba.fit(X_train, y_train).predict_proba(X_test)
probas5_ = ada.fit(X_train, y_train).predict_proba(X_test)
probas6_ = knn.fit(X_train_std, y_train).predict_proba(X_test_std)

In [ ]:
y_test

In [ ]:
fpr1, tpr1, thresholds1 = roc_curve(y_test, probas1_[:, 1])
roc_auc_model1 = auc(fpr1, tpr1)
#fpr2, tpr2, thresholds2 = roc_curve(y_test, probas2_[:, 1])
#roc_auc_model2 = auc(fpr2, tpr2)
fpr3, tpr3, thresholds3 = roc_curve(y_test, probas3_[:, 1])
roc_auc_model3 = auc(fpr3, tpr3)
fpr4, tpr4, thresholds4 = roc_curve(y_test, probas4_[:, 1])
roc_auc_model4 = auc(fpr4, tpr4)
fpr5, tpr5, tresholds5 = roc_curve(y_test, probas5_[:, 1])
roc_auc_model5 = auc(fpr5, tpr5)
fpr6, tpr6, thresholds6 = roc_curve(y_test, probas6_[:, 1])
roc_auc_model6 = auc(fpr6, tpr6)
print("AUC for Logistic Regression Model : ",roc_auc_model1)
#print("AUC for SVM Model:", roc_auc_model2)
print("AUC for Random Forest Model :" ,roc_auc_model3)
print("AUC for bagging model :", roc_auc_model4)
print("AUC for ada model :", roc_auc_model5)
print("AUC for knn model :", roc_auc_model6)

### error: muticlass format is not supported

In [ ]:
plt.clf()
plt.plot(fpr1, tpr1, label='Logistic Model (area = %0.2f)' % roc_auc_model1)
#plt.plot(fpr2, tpr2, label='SVC Model (area = %0.2f)' % roc_auc_model2)
plt.plot(fpr3, tpr3, label='Random Forest Model (area = %0.2f)' % roc_auc_model3)
plt.plot(fpr4, tpr4, label='bagging Model (area = %0.2f)' % roc_auc_model4)
plt.plot(fpr5, tpr5, label='ada Model (area = %0.2f)' % roc_auc_model5)
plt.plot(fpr6, tpr6, label='knn Model (area = %0.2f)' % roc_auc_model6)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiverrating characteristic example')
plt.legend(loc="lower right")
plt.show()

# L1 규제

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.subplot(111)
    
colors = ['blue', 'green', 'red', 'cyan', 
          'magenta', 'yellow', 'black', 
          'pink', 'lightgreen', 'lightblue', 
          'gray', 'indigo', 'orange']

weights, params = [], []
for c in np.arange(-4., 6.):
    lr3 = LogisticRegression(penalty='l1', C=10.**c, solver='liblinear', 
                            multi_class='ovr', random_state=0)
    lr3.fit(X_train_std, y_train)
    weights.append(lr3.coef_[1])
    params.append(10**c)

weights = np.array(weights)

for column, color in zip(range(weights.shape[1]), colors):
    plt.plot(params, weights[:, column],
             label=wine.columns[column + 1],
             color=color)
plt.axhline(0, color='black', linestyle='--', linewidth=3)
plt.xlim([10**(-5), 10**5])
plt.ylabel('weight coefficient')
plt.xlabel('C')
plt.xscale('log')
plt.legend(loc='upper left')
ax.legend(loc='upper center', 
          bbox_to_anchor=(1.38, 1.03),
          ncol=1, fancybox=True)
#plt.savefig('images/04_07.png', dpi=300, 
#            bbox_inches='tight', pad_inches=0.2)
plt.show()

# 특성중요도

In [ ]:
from sklearn.ensemble import RandomForestClassifier

feat_labels = wine.columns[0:14]

forest = RandomForestClassifier(n_estimators=500,
                                random_state=1)

forest.fit(X_train, y_train)
importances = forest.feature_importances_

indices = np.argsort(importances)[::-1]

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))

plt.title('Feature Importance')
plt.bar(range(X_train.shape[1]), 
        importances[indices],
        align='center')

plt.xticks(range(X_train.shape[1]), 
           feat_labels[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
#plt.savefig('images/04_09.png', dpi=300)
plt.show()

In [ ]:
from sklearn.feature_selection import SelectFromModel

sfm = SelectFromModel(forest, threshold=0.08, prefit=True)
X_selected = sfm.transform(X_train)
print('Number of features that meet this threshold criterion:', 
      X_selected.shape[1])

In [ ]:
for f in range(X_selected.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, 
                            feat_labels[indices[f]], 
                            importances[indices[f]]))